# Long e Short - Cointegração
- Irá pegar uma combinação de pares cointegrados, anteriormente encontrados, para gerar um BD em CSV
- Retornará sobre cada par o coeficiente angular e o ponto de intersecção 
- Será usado para no Excel acompanhar e ter retorno quando o par estiver acima ou abaixo de 2 desvios padrões

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.tsa.stattools as ts
from datetime import date
from arch.unitroot import ADF

import yfinance as yf 

import warnings
warnings.filterwarnings("ignore")

In [2]:
lista_ativos = list(pd.read_csv('https://raw.githubusercontent.com/Lessalc/Long-Short/main/ativos.txt'))
char_excluido = -3

In [3]:
coluna_df = ['Ativo_Independente', 'Ativo_Dependente', 'CoefAng -100p', 'Inters -100p', 'med_res -100p', 'desv_res -100p', 'Adf-Stats-100p', 
             'CoefAng -120p', 'Inters -120p', 'med_res -120p', 'desv_res -120p', 'Adf-Stats-120p',
             'CoefAng -140p', 'Inters -140p', 'med_res -140p', 'desv_res -140p', 'Adf-Stats-140p',
             'CoefAng -160p', 'Inters -160p', 'med_res -160p', 'desv_res -160p', 'Adf-Stats-160p',
             'CoefAng -180p', 'Inters -180p', 'med_res -180p', 'desv_res -180p', 'Adf-Stats-180p',
             'CoefAng -200p', 'Inters -200p', 'med_res -200p', 'desv_res -200p', 'Adf-Stats-200p',
             'CoefAng -220p', 'Inters -220p', 'med_res -220p', 'desv_res -220p', 'Adf-Stats-220p',
             'CoefAng -250p', 'Inters -250p', 'med_res -250p', 'desv_res -250p', 'Adf-Stats-250p']
df = pd.DataFrame(columns = coluna_df)

In [4]:
ativos = pd.read_csv('https://raw.githubusercontent.com/Lessalc/Long-Short/main/Pares_Cointegrados%20-%20Selecionados.csv')
ativos.head()

,Ativo_Independente,Ativo_Dependente,ADF-100,ADF-120,ADF-140,ADF-160,ADF-180,ADF-200,ADF-220,ADF-250,Total
0,ABEV3,BBDC3,0,0,0,[1.40452359],[1.47593535],[1.4653124],[1.48454333],[1.74717865],5
1,ABEV3,BBDC4,0,[1.8517902],[1.82796572],[1.82365818],[1.88151277],[1.82937519],[1.81092745],[1.9602221],7
2,ABEV3,BRDT3,[0.50565028],0,[0.39777181],[0.39556566],[0.45217708],[0.69523932],0,0,5
3,ABEV3,CMIG4,[1.09244292],[1.0740501],0,0,[1.10318534],[1.12846237],[1.17399626],[1.12552004],6
4,ABEV3,CPFE3,0,[0.79154024],[0.76856643],[0.75994987],[0.71356694],[0.86328654],0,[1.29036957],6


In [5]:
ativos_array = np.array([ativos['Ativo_Independente'].values, ativos['Ativo_Dependente'].values])

In [6]:
dados = yf.download(tickers = lista_ativos, period='2y', interval='1d', auto_adjust=True)

[*********************100%***********************]  83 of 83 completed

1 Failed download:
- UNNAMED: 82: No data found, symbol may be delisted


In [7]:
dados = dados['Close']
dados = dados.loc[dados['^BVSP'].notna(), :]
dados.dropna(axis=1, how='any', inplace=True)
dados.dropna(axis=0, how='all', inplace=True)

### Atenção - O próximo comando irá demorar um tempo.
- A máquina vai fazer o teste ADF em uma Combinação de todos os pares cointegrados

In [8]:
for i in range(len(ativos)):
    
    ativo_x = dados.loc[:, str(ativos_array[0,i]+'.SA')]

    ativo_y = dados.loc[:, str(ativos_array[1,i]+'.SA')]
    
    
    par = pd.merge(ativo_x, ativo_y, how='inner', on='Date')
    
    df.loc[i, 'Ativo_Independente'] = ativos_array[0,i]
    df.loc[i, 'Ativo_Dependente'] = ativos_array[1,i]
    
    l_ini = len(par.index)
    intervalo = [100,120,140,160,180,200,220,250]
    k = 2

    
    for n in intervalo:
        par_n = par.iloc[l_ini-n:,:]
        
        X = par_n.iloc[:, 0].values
        y = par_n.iloc[:, 1].values
        
        X = X.reshape(-1, 1)
        modelo = LinearRegression()
        modelo.fit(X,y)
        
        y_pred = modelo.predict(X)
        
        residuos = y - y_pred
        
        adf = ADF(residuos)
                
        if adf.stat < adf.critical_values['5%']:
            df.iloc[i,k] = float(modelo.coef_)
            k += 1
            df.iloc[i,k] = float(modelo.intercept_)
            k += 1
            df.iloc[i,k] = np.mean(residuos)
            k += 1
            df.iloc[i,k] = np.std(residuos)
            k += 1
            if adf.stat < adf.critical_values['1%']:
                df.iloc[i,k] = '99'
            else: df.iloc[i,k] = '95'
            k += 1
        else:
            df.iloc[i,k] = 0
            k += 1
            df.iloc[i,k] = 0
            k += 1
            df.iloc[i,k] = 0
            k += 1
            df.iloc[i,k] = 0
            k += 1
            df.iloc[i,k] = '<90'
            k += 1


In [9]:
df.head()

,Ativo_Independente,Ativo_Dependente,CoefAng -100p,Inters -100p,med_res -100p,desv_res -100p,Adf-Stats-100p,CoefAng -120p,Inters -120p,med_res -120p,...,CoefAng -220p,Inters -220p,med_res -220p,desv_res -220p,Adf-Stats-220p,CoefAng -250p,Inters -250p,med_res -250p,desv_res -250p,Adf-Stats-250p
0,ABEV3,BBDC3,0,0,0,0,<90,0,0,0,...,1.48454,-0.0230455,3.10055e-15,0.906853,99,1.74718,-3.25309,1.84741e-16,1.22194,99
1,ABEV3,BBDC4,0,0,0,0,<90,1.85179,-2.66311,2.39808e-15,...,1.81093,-2.39359,-1.2596e-15,1.02451,99,1.96022,-4.15569,-8.52651e-16,1.2181,95
2,ABEV3,BRDT3,0.50565,14.0628,-1.03029e-15,0.878646,95,0,0,0,...,0,0,0,0,<90,0,0,0,0,<90
3,ABEV3,CMIG4,1.09244,-3.3274,-1.1724e-15,0.555771,95,1.07405,-3.05576,-1.18424e-16,...,1.174,-4.82709,-2.58379e-16,0.735144,99,1.12552,-4.18911,-1.79057e-15,0.743232,99
4,ABEV3,CPFE3,0,0,0,0,<90,0.79154,19.115,-1.0066e-15,...,0,0,0,0,<90,1.29037,12.1121,-2.37321e-15,1.56074,95


In [10]:
# Gerando o arquivo CSV
df.to_csv('BD_LS.csv', index=False, header=True)